In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import sys 
import glob
import time
import re
from pathlib import Path

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [3]:
data_directory =  Path('../input/processed-voxels/voxels')

mri_types = ['FLAIR','T1w','T1wCE','T2w']

In [4]:
train_labels = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')

train_labels['BraTS21ID'] = train_labels['BraTS21ID'].apply(lambda x : str(x).zfill(5))
train_labels['BraTS21ID'] = train_labels['BraTS21ID'].astype(str)
train_labels = train_labels[~train_labels['BraTS21ID'].isin(['00109', '00123', '00709'])]
print(train_labels.shape)
train_labels.head()

(582, 2)


,BraTS21ID,MGMT_value
0,00000,1
1,00002,1
2,00003,0
3,00005,1
4,00006,1


In [5]:
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 582 entries, 0 to 584
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   BraTS21ID   582 non-null    object
 1   MGMT_value  582 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 13.6+ KB


In [6]:
df_train, df_valid = train_test_split(
    train_labels, 
    test_size=0.2, 
    random_state=42, 
    stratify=train_labels["MGMT_value"],
)

print(df_train.shape)
print(df_valid.shape)

(465, 2)
(117, 2)


In [7]:
voxel = np.load('../input/processed-voxels/voxels/FLAIR/00000.npy')
voxel.shape

(64, 256, 256)

In [8]:
voxel = np.expand_dims(voxel, 0)
voxel.shape

(1, 64, 256, 256)

In [9]:
mri_type = 'FLAIR'
scan_id = '00002.npy'
data = np.load(data_directory.joinpath(mri_type).joinpath(scan_id))

In [10]:
data_dir = '../input/processed-voxels/voxels'

In [11]:
class MRIScanDataset(Dataset) :
    
    def __init__(self, paths, targets=None, mri_type=None, 
                 label_smoothing=0.01, split="train", augment=False) :
        
        self.paths = paths
        self.targets = targets
        self.mri_type = mri_type
        self.label_smoothing = label_smoothing
        self.split = split
        self.augment = augment
        
        
    def __len__(self) :
        return len(self.paths)
    
    def __getitem__(self, index) :
        
        scan_id = self.paths[index] + '.npy'
        
        if self.targets is None:
            data = np.load(data_directory.joinpath(self.mri_type[0]).joinpath(scan_id))
            data = np.expand_dims(data, 0)
            
        else:
            if self.augment:
                rotation = np.random.randint(0,4)
            else:
                rotation = 0
                
            data = np.load(data_directory.joinpath(self.mri_type[0]).joinpath(scan_id))
            data = np.expand_dims(data, 0)
            
            
        if self.targets is None:
            return {"X": torch.tensor(data).float(), "id": scan_id}
        else:
            y = torch.tensor(abs(self.targets[index]-self.label_smoothing), dtype=torch.float)
            return {"X": torch.tensor(data).float(), "y": y}
               

In [12]:
df_train.head()

,BraTS21ID,MGMT_value
537,00789,1
27,00044,0
508,00740,1
248,00360,1
283,00410,0


In [13]:
df_train.loc[:,"MRI_Type"] = 'FLAIR'
train_dataset = MRIScanDataset(
        df_train["BraTS21ID"].values, 
        df_train["MGMT_value"].values, 
        df_train["MRI_Type"].values,
        augment=False)

train_loader = DataLoader(
        train_dataset,
        batch_size=16,
        shuffle=True,
        num_workers=8,pin_memory = True )


for data in train_loader :
    X1 = data['X']
    y1 = data['y']
    print(X1.shape)
    print(y1.shape)
    break

torch.Size([16, 1, 64, 256, 256])
torch.Size([16])


In [22]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.conv_layer1 = self._conv_layer_set(1, 16)
        self.conv_layer2 = self._conv_layer_set(16, 32)
        self.conv_layer3 = self._conv_layer_set(32, 64)
        self.fc1 = nn.Linear(6*30*30*64, 64) 
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.LeakyReLU()
        self.batch=nn.BatchNorm1d(64)
        self.drop=nn.Dropout(p=0.15)        
        
    def _conv_layer_set(self, in_c, out_c):
        conv_layer = nn.Sequential(
        nn.Conv3d(in_c, out_c, kernel_size=(3, 3, 3), padding=0),
        nn.BatchNorm3d(out_c),
        nn.LeakyReLU(),
        nn.MaxPool3d((2, 2, 2)),
        nn.Dropout3d(0.2)
        )
        return conv_layer
    

    def forward(self, x):
        # Set 1
#         print(x.shape)
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.conv_layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.batch(out)
        out = self.drop(out)
        out = self.fc2(out)
        
        return out

In [23]:
class TrainModel:
    
    def __init__(self, model, device, 
                optimizer, criterion ):
        
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion
        
        self.best_valid_score = np.inf
        self.n_patience = 0
        self.lastmodel = None
        
    def fit(self, epochs, train_dataloader, valid_dataloader, save_path, patience):
        
        start_time = time.time()
        
        for epoch in range(1, epochs + 1):
            
            print(f'Running Epoch {epoch}...................')
            
            train_loss, train_auc = self.train(train_dataloader)
            val_loss, val_auc = self.validation(valid_dataloader)
            
            print(f'For Epoch {epoch :>7d} Train Loss {train_loss : >5f} Train AUC {train_auc} Val Loss {val_loss} Val AUC {val_auc} ')
            print(f'For Epoch {epoch :>7d} Time Taken {(time.time() - start_time)/60}')
            
            
            if self.best_valid_score > val_loss: 
                
                self.save_model(epoch, save_path, val_loss, val_auc)
                print(f'AUC Improved from {self.best_valid_score :4f} to {val_loss}. Saved model to {self.lastmodel}')
                
                #updating the lossed
                self.best_valid_score = val_loss
                self.n_patience = 0
            else:
                self.n_patience += 1
            
            if self.n_patience >= patience:
                print(f"\nValid auc didn't improve last {patience} epochs.")
                break
            
            
            
    def train(self, train_dataloader) :
        '''
        For Training the model.
        We will be calculating batch wise loss and 
        finally calcualting auc on the overall y
        '''
    
        self.model.train()
        sum_loss = 0
        y_all = []
        output_all = []
        start_time = time.time()

        for batch, data in enumerate(train_dataloader) :

            X = data['X'].to(self.device)
            y = data['y'].to(self.device)

            self.optimizer.zero_grad()     #clearning the accumulated gradients
            pred = self.model(X).squeeze(1)           #make the prediction
            loss = self.criterion(pred, y) #calcualte the loss
            loss.backward()           #backpropagation
            self.optimizer.step()          #update weights

            sum_loss += loss.detach().item()
            y_all.extend(data['y'].tolist()) #save all y values to y_val
            output_all.extend(torch.sigmoid(pred).tolist())  #save all pred to output all

            #print peformance
            if batch % 20 == 0 :
                time_taken = (time.time() - start_time)
                start_time = time.time()
                print(f'Train Batch {batch + 1 :>7d} Loss : {sum_loss/(batch +1)} Time Taken : {time_taken/60} ')

        y_all = [1 if x > 0.5 else 0 for x in y_all]
        train_auc = roc_auc_score(y_all, output_all)

        return sum_loss/len(train_dataloader) , train_auc
    
    
    
    def validation(self, val_dataloader) :
    
        self.model.eval()
        sum_loss = 0
        y_all = []
        output_all = []

        for batch, data in enumerate(val_dataloader) :

            with torch.no_grad() :

                X_val = data['X'].to(self.device)
                y_val = data['y'].to(self.device)

                pred = self.model(X_val).squeeze(1)   #make the prediction
                loss = self.criterion(pred, y_val) #calcualte the loss

                sum_loss += loss.detach().item()
                y_all.extend(data['y'].tolist()) #save all y values to y_val
                output_all.extend(torch.sigmoid(pred).tolist())  #save all pred to output all

                #print peformance
                if batch % 20 == 0 :
                    print(f'Test Batch {batch + 1 :>7d} Loss : {sum_loss/(batch +1)}')

        y_all = [1 if x > 0.5 else 0 for x in y_all]
        val_auc = roc_auc_score(y_all, output_all)

        return sum_loss/len(val_dataloader) , val_auc

            
    def save_model(self, n_epoch, save_path, loss, auc):
        
        self.lastmodel = f"{save_path}_loss{loss:.3f}_auc{auc:.3f}.pth"
        torch.save(self.model.state_dict(), self.lastmodel)
       

In [28]:
#train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_mri_type(df_train, df_valid, mri_type):
    
    #creating data based on selected mri_type
    if mri_type=="all":
        train_list = []
        valid_list = []
        for mri_type in mri_types:
            df_train.loc[:,"MRI_Type"] = mri_type
            train_list.append(df_train.copy())
            df_valid.loc[:,"MRI_Type"] = mri_type
            valid_list.append(df_valid.copy())

        df_train = pd.concat(train_list)
        df_valid = pd.concat(valid_list)
        
    else:
        df_train.loc[:,"MRI_Type"] = mri_type
        df_valid.loc[:,"MRI_Type"] = mri_type

    print(df_train.shape, df_valid.shape)
    display(df_train.head())
    
    #train dataset
    train_dataset = MRIScanDataset(
        df_train["BraTS21ID"].values, 
        df_train["MGMT_value"].values, 
        df_train["MRI_Type"].values,
        augment=False)
    
    #valid dataset
    valid_dataset = MRIScanDataset(
        df_valid["BraTS21ID"].values, 
        df_valid["MGMT_value"].values,
        df_valid["MRI_Type"].values)
    
    #train dataloader
    train_loader = DataLoader(
        train_dataset,
        batch_size=8,
        shuffle=True,
    drop_last=True)

    valid_loader = DataLoader(
        valid_dataset, 
        batch_size=8,
        shuffle=False,
    drop_last=True)
    
    #load model
    model = CNNModel()
    model.to(device)
    
    #define optimizer & criterion
    criterion = F.binary_cross_entropy_with_logits
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
    

    trainer = TrainModel(model, device, 
                    optimizer,criterion)

    history = trainer.fit(epochs= 10, train_dataloader= train_loader, valid_dataloader= valid_loader, save_path= f"{mri_type}", patience= 6)
    
    return trainer.lastmodel


In [29]:
print(device)

cuda


In [30]:
modelfiles = None

if not modelfiles:
    modelfiles = [train_mri_type(df_train, df_valid, m) for m in ['T2w', 'FLAIR', 'T1wCE', 'T1w']]
    print(modelfiles)

(465, 3) (117, 3)


,BraTS21ID,MGMT_value,MRI_Type
537,00789,1,T2w
27,00044,0,T2w
508,00740,1,T2w
248,00360,1,T2w
283,00410,0,T2w


Running Epoch 1...................
Train Batch       1 Loss : 0.7419384717941284 Time Taken : 0.012706617514292399 
Train Batch      21 Loss : 0.9331856994401841 Time Taken : 0.234485658009847 
Train Batch      41 Loss : 0.8745185747379209 Time Taken : 0.23652841250101725 
Test Batch       1 Loss : 0.6282409429550171
For Epoch       1 Train Loss 0.822478 Train AUC 0.532935916542474 Val Loss 0.6924331571374621 Val AUC 0.542948717948718 
For Epoch       1 Time Taken 0.7594892501831054
AUC Improved from  inf to 0.6924331571374621. Saved model to T2w_loss0.692_auc0.543.pth
Running Epoch 2...................
Train Batch       1 Loss : 0.811424732208252 Time Taken : 0.011146438121795655 
Train Batch      21 Loss : 0.7513163032985869 Time Taken : 0.23562888701756796 
Train Batch      41 Loss : 0.7253739121483593 Time Taken : 0.23792458375295003 
Test Batch       1 Loss : 0.6169658899307251
For Epoch       2 Train Loss 0.723236 Train AUC 0.5249241196953616 Val Loss 0.6697958069188255 Val AUC 0

,BraTS21ID,MGMT_value,MRI_Type
537,00789,1,FLAIR
27,00044,0,FLAIR
508,00740,1,FLAIR
248,00360,1,FLAIR
283,00410,0,FLAIR


Running Epoch 1...................
Train Batch       1 Loss : 0.8240934610366821 Time Taken : 0.012363346417744954 
Train Batch      21 Loss : 1.019320206982749 Time Taken : 0.2835907697677612 
Train Batch      41 Loss : 0.896081475222983 Time Taken : 0.28166722853978476 
Test Batch       1 Loss : 0.6079650521278381
For Epoch       1 Train Loss 0.861728 Train AUC 0.47554023845007454 Val Loss 0.6660994589328766 Val AUC 0.6557692307692308 
For Epoch       1 Time Taken 0.961834458510081
AUC Improved from  inf to 0.6660994589328766. Saved model to FLAIR_loss0.666_auc0.656.pth
Running Epoch 2...................
Train Batch       1 Loss : 0.7887693643569946 Time Taken : 0.01188277800877889 
Train Batch      21 Loss : 0.7139572813397362 Time Taken : 0.24015438556671143 
Train Batch      41 Loss : 0.7180274361517371 Time Taken : 0.23812594413757324 
Test Batch       1 Loss : 0.6212069988250732
For Epoch       2 Train Loss 0.711594 Train AUC 0.5261922503725782 Val Loss 0.6584216015679496 Val AU

,BraTS21ID,MGMT_value,MRI_Type
537,00789,1,T1wCE
27,00044,0,T1wCE
508,00740,1,T1wCE
248,00360,1,T1wCE
283,00410,0,T1wCE


Running Epoch 1...................
Train Batch       1 Loss : 0.6425114274024963 Time Taken : 0.018671456972757974 
Train Batch      21 Loss : 1.1026231674920945 Time Taken : 0.32405279874801635 
Train Batch      41 Loss : 0.9532143713497534 Time Taken : 0.3180151621500651 
Test Batch       1 Loss : 0.7387741804122925
For Epoch       1 Train Loss 0.899050 Train AUC 0.509152188890751 Val Loss 0.7054603866168431 Val AUC 0.4342948717948718 
For Epoch       1 Time Taken 1.0644724170366924
AUC Improved from  inf to 0.7054603866168431. Saved model to T1wCE_loss0.705_auc0.434.pth
Running Epoch 2...................
Train Batch       1 Loss : 0.7883315086364746 Time Taken : 0.011957859992980957 
Train Batch      21 Loss : 0.722917358080546 Time Taken : 0.23712121645609538 
Train Batch      41 Loss : 0.7004657358658023 Time Taken : 0.23841469685236613 
Test Batch       1 Loss : 0.6065022945404053
For Epoch       2 Train Loss 0.706465 Train AUC 0.577304806062976 Val Loss 0.6885048917361668 Val AU

,BraTS21ID,MGMT_value,MRI_Type
537,00789,1,T1w
27,00044,0,T1w
508,00740,1,T1w
248,00360,1,T1w
283,00410,0,T1w


Running Epoch 1...................
Train Batch       1 Loss : 0.7350436449050903 Time Taken : 0.02062479257583618 
Train Batch      21 Loss : 0.9648349341892061 Time Taken : 0.33114058574040733 
Train Batch      41 Loss : 0.903342340050674 Time Taken : 0.332040810585022 
Test Batch       1 Loss : 0.6534786820411682
For Epoch       1 Train Loss 0.869699 Train AUC 0.509915647170549 Val Loss 0.6808912839208331 Val AUC 0.5958333333333333 
For Epoch       1 Time Taken 1.1046905318895976
AUC Improved from  inf to 0.6808912839208331. Saved model to T1w_loss0.681_auc0.596.pth
Running Epoch 2...................
Train Batch       1 Loss : 0.5835310220718384 Time Taken : 0.012427397569020589 
Train Batch      21 Loss : 0.7399781431470599 Time Taken : 0.2705345312754313 
Train Batch      41 Loss : 0.7244357059641582 Time Taken : 0.28838709990183514 
Test Batch       1 Loss : 0.6939533352851868
For Epoch       2 Train Loss 0.710550 Train AUC 0.52172131910694 Val Loss 0.7137791727270398 Val AUC 0.42

In [52]:
def predict(modelfile, df, mri_type, split):
    
    print("Predict:", modelfile, mri_type, df.shape)
    df.loc[:,"MRI_Type"] = mri_type
    
    data_retriever = MRIScanDataset(
        df.index.values, 
        mri_type=df["MRI_Type"].values,
        split=split
    )

    data_loader = DataLoader(
        data_retriever,
        batch_size=8,
        shuffle=False,
        num_workers=8,
    )
   
    model = CNNModel()
    model.to(device)
    
    model.load_state_dict(torch.load(modelfile))
    model.eval()
    
    y_pred = []
    ids = []

    for e, batch in enumerate(data_loader,1):
        
        print(f"{e}/{len(data_loader)}", end="\r")
        with torch.no_grad():
            tmp_pred = torch.sigmoid(model(batch["X"].to(device))).cpu().numpy().squeeze()
            if tmp_pred.size == 1:
                y_pred.append(tmp_pred)
            else:
                y_pred.extend(tmp_pred.tolist())
            batch_id_list = list(batch["id"])
            batch_id_list = [x.split('.')[0] for x in batch_id_list]
#             print(batch_id_list)
            ids.extend(batch_id_list)
            
    preddf = pd.DataFrame({"BraTS21ID": ids, "MGMT_value": y_pred}) 
    preddf = preddf.set_index("BraTS21ID")
    return preddf

In [53]:
modelfiles

['T2w_loss0.655_auc0.706.pth',
 'FLAIR_loss0.649_auc0.704.pth',
 'T1wCE_loss0.674_auc0.648.pth',
 'T1w_loss0.666_auc0.641.pth']

In [55]:
mri_types = ['T2w', 'FLAIR', 'T1wCE', 'T1w']
# df_valid = df_valid.set_index("BraTS21ID")

for m, mtype in zip(modelfiles,  mri_types):
    pred = predict(m, df_valid, mtype, "train")
    df_valid[f"MGMT_pred_{mtype}"] = pred
    
df_valid.head()

Predict: T2w_loss0.655_auc0.706.pth T2w (117, 6)
Predict: FLAIR_loss0.649_auc0.704.pth FLAIR (117, 6)
Predict: T1wCE_loss0.674_auc0.648.pth T1wCE (117, 6)
Predict: T1w_loss0.666_auc0.641.pth T1w (117, 6)


,MGMT_value,MRI_Type,MGMT_pred_T2w,MGMT_pred_FLAIR,MGMT_pred_T1wCE,MGMT_pred_T1w
BraTS21ID,,,,,,
00418,0,T1w,0.477596,0.644198,0.335624,0.530639
00836,0,T1w,0.457938,0.426843,0.484292,0.544154
00705,1,T1w,0.493994,0.585502,0.542805,0.567514
00078,1,T1w,0.598755,0.579366,0.535969,0.599698
00062,1,T1w,0.670807,0.601613,0.534775,0.559082


In [56]:
df_valid['avg_pred'] = df_valid[['MGMT_pred_FLAIR', 'MGMT_pred_T1w', 'MGMT_pred_T1wCE', 'MGMT_pred_T2w']].mean(axis =1)
roc_auc_score(df_valid['MGMT_value'], df_valid['avg_pred'])

0.7249266862170087

In [57]:
weights = [0.706, 0.704, 0.648, 0.641]
weights = [i/sum(weights) for i in weights]
weights

[0.2615783623564283,
 0.2608373471656169,
 0.2400889218228974,
 0.23749536865505744]

In [58]:
df_valid['weight_avg_pred'] = df_valid['MGMT_pred_T2w'] * weights[0] + df_valid['MGMT_pred_FLAIR'] * weights[1] + df_valid['MGMT_pred_T1wCE'] * weights[2] + df_valid['MGMT_pred_T1w'] * weights[3]
    
roc_auc_score(df_valid['MGMT_value'], df_valid['weight_avg_pred'])

0.7243401759530791

In [ ]:
submission = pd.read_csv(f"{data_directory}/sample_submission.csv", index_col="BraTS21ID")

for m, mtype in zip(modelfiles,  mri_types):
    pred = predict(m, submission, mtype, split="test")
    submission[f"MGMT_pred_{mtype}"] = pred
    
submission.head()

In [ ]:
submission['MGMT_value'] = submission[['MGMT_pred_FLAIR', 'MGMT_pred_T1w', 'MGMT_pred_T1wCE', 'MGMT_pred_T2w']].mean(axis =1)
submission['MGMT_value'].to_csv('submission.csv')